# Métodos formales en computación

**Alumno: Javier Ruiz**

**Asignatura: UNIT 34 ANALYTICAL METHODS**

### Primera parte

### Correción mediante la lógica de Hoare

La lógica de Hoare es un sistema formal que permite razonar sobre la corrección de los programas. Se basa en tripletas de la forma {P} C {Q}, donde:

$P$ es la precondición
$C$ es el comando o programa
$Q$ es la postcondición

**Precondición y Postcondición**

Para nuestro programa FindMax:

- Precondición ($P$): $n \geq 1 \land A \text{ es un array de } n \text{ enteros}$

- Postcondición ($Q$): $max = \max{A[i] : 0 \leq i < n}$

Que podemos expresar también como:

$(\forall i: 0 \leq i < n: max \geq A[i]) \land (\exists j: 0 \leq j < n: max = A[j])$

### Verificación de corrección

Utilizaremos un invariante de bucle para demostrar la corrección:

**Invariante de bucle ($I$):**

$I \equiv (1 \leq i \leq n+1) \land (max = \max{A[j] : 0 \leq j < i})$

O en notación lógica: $I \equiv (1 \leq i \leq n+1) \land (\forall j: 0 \leq j < i: max \geq A[j]) \land (\exists k: 0 \leq k < i: max = A[k])$

**Verificación de inicialización**

Antes del bucle: $i = 1$ y $max = A[0]$

Tenemos que verificar que $I$ se cumple:

$1 \leq 1 \leq n+1$ 

$max = A[0] = \max{A[j] : 0 \leq j < 1}$ 

**Verificación de mantenimiento del invariante**

Suponemos que $I$ es cierto al inicio de una iteración. Debemos verificar que sigue siendo cierto al final:

Si $A[i] > max$, entonces $max := A[i]$:

$max' = A[i] = \max{A[j] : 0 \leq j \leq i}$ 

Si $A[i] \leq max$, entonces $max$ no cambia:

$max' = max = \max{A[j] : 0 \leq j < i} = \max{A[j] : 0 \leq j \leq i}$ 

Después, $i$ se incrementa a $i+1$, manteniendo $1 \leq i \leq n+1$.

**Verificación de la terminación**

Cuando el bucle termina, tenemos $i = n+1$, y el invariante nos dice:

$max = \max{A[j] : 0 \leq j < n+1} = \max{A[j] : 0 \leq j < n}$

Esto es exactamente nuestra postcondición.

### Tripleta de Hoare completa

${n \geq 1 \land A \text{ es un array de } n \text{ enteros}}$

$max := A[0];$

$for\ i := 1\ to\ n\ do$

$\quad {I \land i \leq n}$

$\quad if\ A[i] > max\ then$

$\quad \quad max := A[i]$

$\quad end\ if$

$\quad {I[i \leftarrow i+1]}$

$end\ for$

${max = \max{A[j] : 0 \leq j < n}}$

### Especificación del leguaje Z

El lenguaje Z es un lenguaje formal basado en la teoría de conjuntos y la lógica de primer orden. Usaremos esquemas Z para modelar el sistema:

Donde: 

$FindMaxSystem$ define el estado del sistema

$FindMaxInit$ establece las precondiciones

$FindMaxOperation$ especifica la transformación del estado

$\Delta FindMaxSystem$ indica que el estado cambia

$max'$ es el valor de $max$ después de la operación

### Implementación en Python

In [1]:
def find_max(A):
    """
    Encuentra el valor máximo en un array de enteros.
    Precondición: A es un array no vacío de enteros
    Postcondición: Retorna el valor máximo de A
    """
    if len(A) == 0:
        raise ValueError("El array no puede estar vacío")
        
    max_val = A[0]
    for i in range(1, len(A)):
        if A[i] > max_val:
            max_val = A[i]
    return max_val

# Prueba básica
A = [3, 7, 1, 9, 5, 2]
print(f"Array: {A}")
print(f"Valor máximo: {find_max(A)}")

Array: [3, 7, 1, 9, 5, 2]
Valor máximo: 9


### Verificación con aserciones en Pyhton

Implementaremos una versión del algoritmo que incluye aserciones para verificar el invariante de bucle:

In [2]:
def find_max_with_assertions(A):
    """
    Encuentra el valor máximo con aserciones que verifican el invariante.
    """
    assert len(A) > 0, "Precondición: El array no puede estar vacío"
    
    max_val = A[0]
    # Invariante inicial: max_val es el máximo de A[0..0]
    assert max_val == A[0], "Invariante inicial"
    
    for i in range(1, len(A)):
        # Invariante de bucle: max_val es el máximo de A[0..i-1]
        assert all(max_val >= A[j] for j in range(i)), "Invariante del bucle - condición ∀"
        assert any(max_val == A[j] for j in range(i)), "Invariante del bucle - condición ∃"
        
        # Cuerpo del bucle
        if A[i] > max_val:
            max_val = A[i]
            
    # Postcondición: max_val es el máximo de todo A
    assert all(max_val >= A[j] for j in range(len(A))), "Postcondición - condición ∀"
    assert any(max_val == A[j] for j in range(len(A))), "Postcondición - condición ∃"
    
    return max_val

# Probamos con el mismo array
try:
    result = find_max_with_assertions(A)
    print(f"Verificación exitosa. Máximo: {result}")
except AssertionError as e:
    print(f"Error en la verificación: {e}")

Verificación exitosa. Máximo: 9


### Pruebas exhaustibas



In [3]:
def test_find_max():
    """
    Conjunto de pruebas para validar la implementación del algoritmo.
    """
    test_cases = [
        ([1, 2, 3, 4, 5], 5),          # Orden ascendente
        ([5, 4, 3, 2, 1], 5),          # Orden descendente
        ([1, 5, 3, 2, 4], 5),          # Orden mixto
        ([42], 42),                    # Un solo elemento
        ([-1, -2, -3, -4, -5], -1),    # Números negativos
        ([5, 5, 5, 5], 5),             # Elementos repetidos
        ([10**9, 10**8, 10**10], 10**10)  # Números grandes
    ]
    
    for arr, expected in test_cases:
        result = find_max(arr)
        assert result == expected, f"Para {arr}, se esperaba {expected} pero se obtuvo {result}"
    
    # Verificar que se lanza una excepción para un array vacío
    try:
        find_max([])
        assert False, "Debería haber lanzado una excepción para array vacío"
    except ValueError:
        pass
    
    print("Todas las pruebas pasaron exitosamente!")

# Ejecutar pruebas
test_find_max()

Todas las pruebas pasaron exitosamente!


### Implementación Z 

Para completar el análisis,  implementar una versión que cumpla con la especificación Z

In [4]:
class FindMaxSystem:
    """Implementación del sistema especificado en Z"""
    
    def __init__(self, A):
        """
        Inicialización del sistema (FindMaxInit)
        """
        self.A = A
        self.n = len(A)
        self.max = None
        
        # Precondición
        assert self.n > 0, "El array no puede estar vacío"
    
    def operation(self):
        """
        Implementación de FindMaxOperation
        """
        # Implementación directa de la especificación Z
        self.max = max(self.A[i] for i in range(self.n))
        return self.max
    
    def implementation(self):
        """
        Implementación algoritmica (la que queremos verificar)
        """
        self.max = self.A[0]
        for i in range(1, self.n):
            if self.A[i] > self.max:
                self.max = self.A[i]
        return self.max
    
    def verify(self):
        """
        Verifica que la implementación satisface la especificación
        """
        # Guardamos el estado actual
        original_max = self.max
        
        # Ejecutamos la especificación
        expected = self.operation()
        
        # Restauramos y ejecutamos la implementación
        self.max = original_max
        actual = self.implementation()
        
        # Verificamos que ambos resultados son iguales
        assert expected == actual, f"Especificación: {expected}, Implementación: {actual}"
        return True

# Probamos la verificación
system = FindMaxSystem(A)
if system.verify():
    print("¡La implementación satisface la especificación formal!")

¡La implementación satisface la especificación formal!
